In [ ]:
import pandas as pd
import networkx as nx
import numpy as np

In [ ]:
data_stations=pd.read_csv("/content/OysterMetro_Complete_Sample.csv")
data_distances=pd.read_excel("/content/Inter station database.xls",'Runtime',header=1)


In [ ]:
df_stations=data_stations[['Destination','Origin']]
df_stations['Destination'] = df_stations['Destination'].str.upper()
df_stations['Origin'] = df_stations['Origin'].str.upper()
df_stations.head()

<ipython-input-100-8079e13b036a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations['Destination'] = df_stations['Destination'].str.upper()
<ipython-input-100-8079e13b036a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations['Origin'] = df_stations['Origin'].str.upper()


,Destination,Origin
0,TOWER HILL,HIGHGATE
1,FINSBURY PARK,MORNINGTON CRESCENT
2,WARWICK AVENUE,PICCADILLY CIRCUS
3,WARREN STREET,CANARY WHARF
4,CALEDONIAN ROAD,LEICESTER SQUARE


In [ ]:
data_distances.columns

Index(['Line', 'Direction', 'Station from (A)', 'Station to (B)',
       'Distance (Kms)', 'Un-impeded Running Time (Mins)',
       'AM peak (0700-1000) Running Time (Mins)',
       'Inter peak (1000 - 1600) Running time (mins)'],
      dtype='object')

In [ ]:
df_distances=data_distances[['Station from (A)','Station to (B)','Distance (Kms)']]
df_distances.head()

,Station from (A),Station to (B),Distance (Kms)
0,HARROW & WEALDSTONE,KENTON,1.74
1,KENTON,SOUTH KENTON,1.40
2,SOUTH KENTON,NORTH WEMBLEY,0.90
3,NORTH WEMBLEY,WEMBLEY CENTRAL,1.27
4,WEMBLEY CENTRAL,STONEBRIDGE PARK,1.71


In [ ]:
# Creating a graph from the distances DataFrame
G = nx.Graph()
for _, row in df_distances.iterrows():
    G.add_edge(row['Station from (A)'], row['Station to (B)'], weight=row['Distance (Kms)'])

# Function to find the shortest path using Dijkstra's algorithm
def find_shortest_path(origin, destination, graph):
    try:
      path = nx.dijkstra_path(graph, origin, destination, weight='weight')
      distance = nx.dijkstra_path_length(graph, origin, destination, weight='weight')
      return path, distance
    except:
      path = "0"
      distance = np.nan
      return path, distance


# Apply the function to each origin-destination pair
results = df_stations.apply(lambda row: find_shortest_path(row['Origin'], row['Destination'], G), axis=1)

# Extracting paths and distances
df_stations['Full_Path'] = results.apply(lambda x: ' -> '.join(x[0]))
df_stations['Total_Distance'] = results.apply(lambda x: x[1])

df_stations


<ipython-input-103-7f3ee7743882>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations['Full_Path'] = results.apply(lambda x: ' -> '.join(x[0]))
<ipython-input-103-7f3ee7743882>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations['Total_Distance'] = results.apply(lambda x: x[1])


,Destination,Origin,Full_Path,Total_Distance
0,TOWER HILL,HIGHGATE,HIGHGATE -> ARCHWAY -> TUFNELL PARK -> KENTISH...,11.98
1,FINSBURY PARK,MORNINGTON CRESCENT,MORNINGTON CRESCENT -> EUSTON (CX) -> WARREN S...,6.88
2,WARWICK AVENUE,PICCADILLY CIRCUS,PICCADILLY CIRCUS -> OXFORD CIRCUS -> REGENTS ...,5.27
3,WARREN STREET,CANARY WHARF,CANARY WHARF -> CANADA WATER -> BERMONDSEY -> ...,10.01
4,CALEDONIAN ROAD,LEICESTER SQUARE,LEICESTER SQUARE -> COVENT GARDEN -> HOLBORN -...,4.43
...,...,...,...,...
70336,VICTORIA,LIMEHOUSE,0,NaN
70337,OLD STREET,BURNT OAK,BURNT OAK -> COLINDALE -> HENDON CENTRAL -> BR...,16.69
70338,STRATFORD,NORTH GREENWICH,NORTH GREENWICH -> CANNING TOWN -> WEST HAM ->...,4.83
70339,HARLESDEN,SOUTH KENTON,SOUTH KENTON -> NORTH WEMBLEY -> WEMBLEY CENTR...,5.41


In [ ]:
df_stations[df_stations['Total_Distance'].notnull()]

,Destination,Origin,Full_Path,Total_Distance
0,TOWER HILL,HIGHGATE,HIGHGATE -> ARCHWAY -> TUFNELL PARK -> KENTISH...,11.98
1,FINSBURY PARK,MORNINGTON CRESCENT,MORNINGTON CRESCENT -> EUSTON (CX) -> WARREN S...,6.88
2,WARWICK AVENUE,PICCADILLY CIRCUS,PICCADILLY CIRCUS -> OXFORD CIRCUS -> REGENTS ...,5.27
3,WARREN STREET,CANARY WHARF,CANARY WHARF -> CANADA WATER -> BERMONDSEY -> ...,10.01
4,CALEDONIAN ROAD,LEICESTER SQUARE,LEICESTER SQUARE -> COVENT GARDEN -> HOLBORN -...,4.43
...,...,...,...,...
70335,CAMDEN TOWN,TOTTENHAM COURT ROAD,TOTTENHAM COURT ROAD -> GOODGE STREET -> WARRE...,3.12
70337,OLD STREET,BURNT OAK,BURNT OAK -> COLINDALE -> HENDON CENTRAL -> BR...,16.69
70338,STRATFORD,NORTH GREENWICH,NORTH GREENWICH -> CANNING TOWN -> WEST HAM ->...,4.83
70339,HARLESDEN,SOUTH KENTON,SOUTH KENTON -> NORTH WEMBLEY -> WEMBLEY CENTR...,5.41


In [ ]:
df_stations.to_csv("new_distance.csv",index=False)
df_stations.to_excel("new_distance.xlsx",index=False)
